**Table of contents**<a id='toc0_'></a>    
- 1. [UE 4268 - QISKIT Project 3](#toc1_)    
- 2. [Introduction](#toc2_)    
- 3. [Defining the chemistry problem](#toc3_)    
  - 3.1. [Hydrogen Molecule](#toc3_1_)    
  - 3.2. [Mapping the problem in the STO-3G basis](#toc3_2_)    
    - 3.2.1. [Spin-orbitals](#toc3_2_1_)    
  - 3.3. [Pauli  representation of the one-electron terms in the Hamiltonian](#toc3_3_)    
  - 3.4. [The Pauli  representation of of the two-electron terms in the Hamiltonian](#toc3_4_)    
  - 3.5. [Reference (Hartree-Fock) States](#toc3_5_)    
  - 3.6. [UCCSD *ansatz*](#toc3_6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[UE 4268 - QISKIT Project 3](#toc0_)

**`vos noms et prenoms`**, `matricule` et `email` 


Department of Physics - Faculty of Science - University of Yaoundé I

`Nom du Laboratoire`

`Date`

$$
\newcommand{\ket}[1]{|#1\rangle} 
\newcommand{\mt}[1]{\mathtt{#1}} 
$$

# 2. <a id='toc2_'></a>[Introduction](#toc0_)

The purpose of this project is to illustrate some of the techniques described in the previous lab sessions, by explicitly demonstrating how to map molecular ground state problems onto a quantum computer. We do this in second quantization for the Hydrogen molecule ($H_2$) in the STO-3G basis. Across the project, we showcase the Jordan-Wigner (JW) and Bravyi-Kitaev (BK) mappings, reduction of active orbitals using the Natural Molecular Orbital (NMO) basis, reduction of qubits using symmetry conservation and the UCC ansatz. Many of these techniques are applicable to ground state and general chemical problems. 

 It is strongly advised to exploit the excellent article of McArdle *et al.* [Quantum computational chemistry](https://doi.org/10.1103/RevModPhys.92.015003) which is freely accessible [here](https://arxiv.org/abs/1808.10402v3).

The other useful articles are:
* Seeley, J. T.; Richard, M. J.; Love, P. J., *The Bravyi-Kitaev Transformation for Quantum Computation of Electronic Structure*, J. Chem. Phys. **137**, 224109 (2012);
* Whitfield, J. D.; Biamonte, J.; Aspuru-Guzik, A., *Simulation of Electronic Structure Hamiltonians Using Quantum Computers*, Mol. Phys. **109**, 735−750 (2011).


# 3. <a id='toc3_'></a>[Defining the chemistry problem](#toc0_)

We address the problem of determining the electronic ground state energy of molecular hydrogen as a function of the distance between the nuclei. This relationship is otherwise known as the ground state energy dissociation curve. 
havehave
<center> 
<img src="./Graphics/H2-STO-6G-The-ground.jpg" width=300 />
 </center>

An accurate description of this energy surface is a key challenge in quantum chemistry that can provide insight on a range of chemical phenomena, e.g., bond breaking and reaction dynamics. To simplify the problem, we apply the Born-Oppenheimer approximation, in which we treat the nuclei as stationary classical particles. This is justified as the ratio of electronic to nuclear mass is roughly 1:1000, leading to a separation in the timescale of their dynamics. The resulting quantum Hamiltonian describing the electronic system, in atomic units, can be written as
\begin{equation}\label{eq:BO-approx}
\mathtt{H}_{\rm elec} =  - \sum_i \frac{\nabla^2_{\mathbf{r}_i}}{2} - \sum_{i,j} \frac{Z_i}{|\mathbf{R}_i - \mathbf{r}_j|} + \sum_{i,j>i} \frac{Z_i Z_j}{|\mathbf{R}_i - \mathbf{R}_j|} + \sum_{i,j>i} \frac{1}{|\mathbf{r}_i - \mathbf{r}_j|},
\end{equation}
where $\mathbf{r}_i$ are the position coordinates of the electrons, which parametrically depend on the fixed position coordinates of the nuclei $\mathbf{R}_i$, and $Z_i$ and $M_i$ denoting the nuclear charges and masses, respectively. The electronic system, as written in the first-quantized picture in Eq. \eqref{eq:BO-approx}, assumes an infinite-dimensional Hilbert space. 

By considering a larger number of orbitals, we are able to recover a larger proportion of the correlation energy in a molecule, resulting in a more accurate estimate of the ground state energy. The figure above  shows the $H_2$ ground state dissociation curves in the STO-3G, 6-31G and cc-PVDZ bases plotted by McArdle *et al.* (2020). We can see that the differences in energy between the three minima are considerably larger than chemical accuracy (1.6mHartree). This highlights the fact that working in a suitably large basis set is crucial for obtaining accurate results.

When applying quantum computation to chemistry, the Hamiltonian is instead often considered in the second-quantized formulation, in which the system can be described approximately using a finite basis. Within this framework, states are labeled by the occupation of the orbitals, and the exchange symmetry of the particles is naturally considered through the use of fermionic creation and annihilation operators. The electronic Hamiltonian can then be expressed in terms of these second-quantized operators as
\begin{equation} \label{eq:H2quant}
\mathtt{H} = \sum_{pq} h_{pq} a^{\dagger}_p a_q + \frac12\sum_{pqrs} h_{pqrs}a^{\dagger}_p a^{\dagger}_q a_r a_s
=\mathtt{H}^{(1)}+\mathtt{H}^{(2)},
\end{equation}
where $a_p$ is an electron annihilation operator that removes an electron from an orbital with label $p$. The weights of the operators are given by the molecular integrals
\begin{align} \label{eq:MolInt}
&h_{pq} = \int \mathrm{d}x \ \phi^*_p(x) \left(\frac{\nabla^2_r}{2} - \sum_i\frac{Z_i}{|\boldsymbol{r}_i - \boldsymbol{r}|}\right) \phi_q(x),
&h_{pqrs} = \int \mathrm{d}x_1 \ \mathrm{d}x_2 \frac{\phi^*_p(x_2) \phi^*_q(x_1)\phi_r(x_1)\phi_s(x_2)}{|\boldsymbol{r}_1-\boldsymbol{r}_2|}, 
\end{align}
where we use $x_i$ to denote the spatial and spin coordinates, i.e., $x_i = ({\boldsymbol{r}}_i, \sigma_i)$. In practice, several electronic structure packages and codes have been developed and optimized for computing these integrals. To prepare our quantum computation for molecular hydrogen, we use such classically pre-computed integrals to prepare the second-quantized Hamiltonian. 


## 3.1. <a id='toc3_1_'></a>[Hydrogen Molecule](#toc0_)

<center> 
 <figure>
<img src="Graphics/h2.png" width="150"/>
  <img src="Graphics/H2-coordinates.png" width=300>
  <figcaption>H2 coordinates. The nuclei are labeled A and B, and the electrons are labeled 1 and 2.</figcaption>
</figure> 
 </center>
 
Hereafter, the symbols $g$ and $u$ will be stood for *gerade* (even) and *ungerade* (odd) with the respect to inversion about the center of mass of the molecule. $\sigma$ will indicate that the projection of the electronic angular momentum about the internuclear axis is zero.

The ground state electronic configuration of $H_2$ is $(1s\sigma_g)^2$ and the Slater determinant for this configuration is
\begin{equation*}
 \psi(1,2) 
  = \frac{1}{\sqrt{2}}\begin{vmatrix}1s\sigma_g(1)\chi_\uparrow(1) & 1s\sigma_g(1)\chi_\downarrow(1) \\ 
  1s\sigma_g(2)\chi_\uparrow(2) & 1s\sigma_g(2)\chi_\downarrow(2) \end{vmatrix}
  = \frac{1}{\sqrt{2}} \ket{1s\sigma_g}\ket{1s\sigma_g}\left(\ket{\uparrow}\ket{\downarrow}-\ket{\downarrow}\ket{\uparrow}\right),
\end{equation*}
with the state function
\begin{align*}
&\ket{1s\sigma_g}=\frac{1}{\sqrt{2(1+S_{AB})}}(\ket{1s_A}+\ket{1s_B}), 
&S_{AB}=\langle 1s_A|1s_B\rangle.
\end{align*}
$S_{AB}$ is the **overlap integral**.


## 3.2. <a id='toc3_2_'></a>[Mapping the problem in the STO-3G basis](#toc0_)


### 3.2.1. <a id='toc3_2_1_'></a>[Spin-orbitals](#toc0_)

The STO-3G basis for $H_2$ includes only the $\{1s\}$ orbital for each hydrogen atom. The $1s$ orbital is represented by a linear combination of three Gaussian functions (GTOs). Each hydrogen atom contributes one spin-orbital, and there are two possible spins for each orbital - resulting in a total of 4 orbitals for STO-3G $H_2$. We denote these orbitals as 
\begin{align*}
	&\ket{1s_{A\uparrow}}, &&\ket{1s_{A\downarrow}}, &&\ket{1s_{B\uparrow}}, &\ket{1s_{B\downarrow}},
\end{align*}
where the subscript $A$ or $B$ denotes which of the two atoms the orbital is centred on, and the $\uparrow,\,\downarrow$ denotes the spin $s_z$ of the electron in the orbital. For convenience, we work in the molecular orbital basis for $H_2$, which is simple to construct manually. These single electron molecular orbitals are given by

 %%  Put your equation here

We can write a Slater determinant in the occupation number basis as
\begin{equation*}
\ket{\psi} = \ket{f_{\sigma_{u \downarrow}}, f_{\sigma_{u \uparrow}}, f_{\sigma_{g \downarrow}}, f_{\sigma_{g \uparrow}}}
\equiv\ket{f_3,f_2,f_1,f_0},
\end{equation*}
where $f_i=1$ if spin-orbital $i$ is occupied, and $f_i=0$ if spin-orbital $i$ is unoccupied. 

We can now calculate the integrals given in Eq. \eqref{eq:MolInt} using these molecular orbitals. These integrals have been calculated for a large number of basis sets, and the results can be obtained by using a computational chemistry package. We must then map the problem Hamiltonian from being written in terms of creation and annihilation operators to being written in terms of qubit operators.

## 3.3. <a id='toc3_3_'></a>[Pauli  representation of the one-electron terms in the Hamiltonian](#toc0_)

We can write the one-electron terms in the Hamiltonian \ref{eq:H2quant} as
\begin{equation*}
\mathtt{H}^{(1)} = h_{00} a^\dagger_0 a_0 + h_{11} a^\dagger_1 a_1 + h_{22} a^\dagger_2 a_2 + h_{33} a^\dagger_3 a_3. 
\end{equation*}

The Table below gives the overlap integrals $h_{pq}$ and $h_{pqrs}$ for molecular hydrogen in a minimal basis obtained through a restricted Hartree-Fock calculation in the PyQuante quantum chemistry package at an internuclear separation of $1.401000$ atomic units ($7.414 \times 10^{-11}$ m).

$$
\begin{array}{|l|r|}\hline
\text{Integrals}& \text{Value (a.u.)}\\\hline
h_{00} = h_{11} &-1.252477 \\\hline
h_{22} = h_{33} &-0.475934 \\\hline
h_{0110} = h_{1001} &  0.674493 \\\hline
h_{2332} = h_{3223} & 0.697397 \\\hline
h_{0220} = h_{0330} = h_{1221} = h_{1331}= h_{2002} = h_{3003} = h_{2112} = h_{3113} & 0.663472 \\\hline
h_{0202} = h_{1313} = h_{2130} = h_{2310} = h_{0312} = h_{0132}  & 0.181287 \\\hline
\end{array}
$$
The one-electron and two-electron integrals for $H_2$ evaluated using molecular spatial orbitals obtained from restricted Hartree-Fock calculation an internuclear separation of $1.401000$ atomic units ($7.414 \times 10^{-11}$ m). See Article of Whitfield et \emph{al.} (2011) and reference therein.

2. Use JW and BK transformations to give the Pauli representation of 
\begin{align*}
&a^\dagger_0a_0, 
&&a^\dagger_1a_1,
&&a^\dagger_2a_2,
&a^\dagger_3a_3.
\end{align*}

3. Use qiskit to find the above JWT and BKT.

1. Use qiskit to check the values of the one-electron and two-electron integrals for $H_2$ given in the Table above.



## 3.4. <a id='toc3_4_'></a>[The Pauli  representation of of the two-electron terms in the Hamiltonian](#toc0_)

Following the work of Whitfield *et al*. \cite{Whitfield2011}, $\mathtt{H}^{(2)}$ simplifies to the following expression for molecular hydrogen in a minimal basis:
\begin{equation*}
	\begin{aligned}
	\mathtt{H}^{(2)} &= h_{0110} a^\dagger_0 a^\dagger_1 a_1 a_0 + h_{2332} a^\dagger_2 a^\dagger_3 a_3 a_2
	 + h_{0330} a^\dagger_0 a^\dagger_3 a_3 a_0 + h_{1221} a^\dagger_1 a^\dagger_2 a_2 a_1 + (h_{0220}- h_{0202}) a^\dagger_0 a^\dagger_2 a_2 a_0 \\
    & + (h_{1331} - h_{1313})a^\dagger_1 a^\dagger_3 a_3 a_1 +h_{0132} (a^\dagger_0 a^\dagger_1 a_3 a_2 + a^\dagger_2a^\dagger_3 a_1 a_0)
    + h_{0312} (a^\dagger_0 a^\dagger_3 a_1 a_2 + a^\dagger_2 a^\dagger_1 a_3a_0).
	\end{aligned}
\end{equation*}
This term in the Hamiltonian is made up of six Coulomb/exchange operators and two double-excitation operators, with the coefficients given by the electron integrals \eqref{eq:MolInt}.

5. Use JW and BK transformations to give the Pauli representation of 
\begin{align*}
&a^\dagger_2 a^\dagger_3 a_3 a_2 ,
&&a^\dagger_0 a^\dagger_3 a_3 a_0 ,
&&a^\dagger_1 a^\dagger_2 a_2 a_1 ,
&&a^\dagger_0 a^\dagger_2 a_2 a_0 ,
&a^\dagger_1 a^\dagger_3 a_3 a_1 .
\end{align*}
6. Use qiskit to find the above JWT and BKT and the Hamiltonian $\mathtt{H}^{(2)}$.


## 3.5. <a id='toc3_5_'></a>[Reference (Hartree-Fock) States](#toc0_)

In the JW encoding, it is simple to construct the Hartree-Fock (HF) state for the $H_2$ molecule. The HF state for $H_2$ in the occupation number basis is given by
\begin{equation*}
	\ket{\psi_{\mathrm{HF}}^{\mathrm{H_2}}} = \ket{0}^{\otimes (M-N_e)}\otimes\ket{1}^{\otimes N_e}=\ket{0011}, 
\end{equation*}
with $M=4$ spin-orbitals and $N_e=2$ electrons. This represents the slater determinant
\begin{equation*}
\psi_{\mathrm{HF}}^{\mathrm{H_2}}(\boldsymbol{r}_1,\boldsymbol{r}_2) = \frac{1}{\sqrt{2}} (\sigma_{g \uparrow}(\boldsymbol{r}_1) \sigma_{g \downarrow}(\boldsymbol{r}_2) - \sigma_{g \uparrow}(\boldsymbol{r}_2) \sigma_{g \downarrow}(\boldsymbol{r}_1) ), 
\end{equation*}
where $\boldsymbol{r}_i$ is the position of electron $i$. The most general state for $H_2$, with the same spin $s_z$ and electron number as the HF state, is given by
\begin{equation*}
	\ket{\psi^{\mathrm{H_2}}} = \alpha\ket{0011} + \beta \ket{1100} + \gamma \ket{1001} + \delta \ket{0110},
\end{equation*}
and the ground state of the $H_2$ molecule at its equilibrium bond distance is given by McArdle et *al* (2020)
\begin{equation*}\label{eq:H2MolEq}
	\ket{\psi_g^{\mathrm{H_2}}} = 0.9939 \ket{0011} - 0.1106 \ket{1100}.
\end{equation*}
The first determinant in the ground state function is the HF state for $H_2$, showing that a mean-field solution is a good approximation for this molecule at this interatomic distance. The second determinant represents the antibonding state, and accounts for dynamical correlation between the electrons due to their electrostatic repulsion. While the HF determinant dominates at the equilibrium separation, at large separation the two determinants contribute equally to the state function. This is because the bonding and antibonding configurations become degenerate. We require both determinants to accurately describe the state, ensuring that only one electron locates around each atom. This is an example of static correlation, which can also be dealt with using multiconfigurational self-consistent field (MCSCF) methods.

## 3.6. <a id='toc3_6_'></a>[UCCSD *ansatz*](#toc0_)

In order to find the ground state of the $H_2$ molecule using the VQE, we need to construct the state on the quantum computer. This can be done using adiabatic state preparation, or using an ansatz. Here we explicitly derive the UCCSD *ansatz* for $H_2$. The UCCSD operator we seek to realise is given by
\begin{align*}
&\mathtt{U} = e^{(\mathtt{T}_1 - \mathtt{T}_1^\dagger) + (\mathtt{T}_2 - \mathtt{T}_2^\dagger)},  
&&\mathtt{T}_1 =  \sum_{i \in\rm{unocc}\atop \alpha \in\rm{occ}} t_{i \alpha} a^\dagger_i a_\alpha,  
&\mathtt{T}_2 = \sum_{i, j \in\rm{unocc}\atop \alpha, \beta \in\rm{occ}} t_{i j \alpha \beta}  a^\dagger_i a^\dagger_j a_\alpha a_\beta, 
\end{align*}
where occ are initially occupied orbitals in the HF state, unocc are initially unoccupied orbitals in the HF state, and $t_{i \alpha}$ and $t_{ij\alpha \beta}$ are variational parameters to be optimised using VQE. For $H_2$, the only operators which do not change the spin $s_z$ of the molecule when acting upon the HF state are: $a^\dagger_2 a_0,~a^\dagger_3 a_1,~a^\dagger_3 a^\dagger_2 a_1 a_0$.

7. Check that by applying those operators on $\ket{0011}$.

Other valid operators are equivalent to these operators, and can be combined with them, such as $a^\dagger_3 a^\dagger_0 a_1 a_0 = -a^\dagger_3 a_1$.

8. Also check this.

As a result, the UCCSD operator takes the form
\begin{equation*}
\mathtt{U} = e^{t_{02}(a^\dagger_2 a_0 -a^\dagger_0 a_2) + t_{13}(a^\dagger_3 a_1 -a^\dagger_1 a_3) + t_{0123}(a^\dagger_3a^\dagger_2 a_1 a_0 -a^\dagger_0a^\dagger_1 a_2 a_3)}. 
\end{equation*}
We can split this operator using Trotterization with a single Trotter step:
\begin{equation}\label{eq:TrotterUCC}\tag{4}
\mathtt{U} =e^{t_{02}(a^\dagger_2 a_0 - a^\dagger_0 a_2)} \times e^{t_{13}(a^\dagger_3 a_1 - a^\dagger_1 a_3)} 
\times e^{t_{0123}(a^\dagger_3 a^\dagger_2 a_1 a_0 - a^\dagger_0 a^\dagger_1 a_2 a_3)}.
\end{equation}

9. Use the JW encoding to the Pauli representation of
\begin{align*}
&(a^\dagger_2 a_0 - a^\dagger_0 a_2),
&&(a^\dagger_3 a_1 - a^\dagger_1 a_3),
&(a^\dagger_3 a^\dagger_2 a_1 a_0 - a^\dagger_0 a^\dagger_1 a_2 a_3)
\end{align*}

It was shown that all Pauli terms arising from the same excitation operators commute.

10. Use the formula $e^{\mathtt{A} t+\mathtt{B} t}=e^{\frac{\mathtt{A}}{2}t}e^{\mathtt{B} t}e^{\frac{\mathtt{A}}{2}t}+\mathcal{O}(t^3)$ to separate each of the exponentials the operator $\mathtt{U}$ in Eq. \eqref{eq:TrotterUCC}) into a product of exponentials of a single Pauli string. 


The UCCSD operator is simplified to 
\begin{equation}\label{eq:H2UCCSD}
\mathtt{U} = e^{-i \theta \mathtt{X}_3 \mathtt{X}_2 \mathtt{X}_1 \mathtt{Y}_0} .
\end{equation}

11. Apply the simplified UCCSD operator to the HF state in $\ket{0011}$ to find 
\begin{equation*}
\mathtt{U}\ket{0011}=\cos\theta\ket{0011}-\sin\theta\ket{1100}
\end{equation*}
Which can reproduce the ground state given by Eq. \eqref{eq:H2MolEq}.

The UCCSD operator can be implemented using the following circuit 

![H2 UCCSD Circuit](./Graphics/H2_CUCCSD_Circuit.jpeg)

The $\mathtt{RX}(\frac{\pi}{2})$ and $\mathtt{W}$ gates rotate the basis such that the exponentiated operator applied to the corresponding qubit is either $\mathtt{Y}$ or $\mathtt{X}$, respectively. Single excitation terms are implemented with a change of basis.

12. Use qiskit to represent that circuit.

13. Use qiskit nature (VQE) to evaluate ground state of the Hydrogen molecule with the above *ansatz*.


In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table